In [273]:
import bibtexparser

bib_file = 'export-bibtex.bib'

with open(bib_file) as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)


In [275]:
len(bib_database.entries)

5

In [277]:
def create_buttons(bib_entry):
    
    links_html = ''
    
    if bib_entry['pdf'] != "":
        links_html += f"""
        <a href="{bib_entry['pdf']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">PDF</a>
        """
    if bib_entry['code'] != "":
        links_html += f"""
        <a href="{bib_entry['code']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">Code</a>
        """
    if bib_entry['project_page'] != "":
        links_html += f"""
        <a href="{bib_entry['project_page']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">Project Page</a>
        """
    if bib_entry['source'] != "":
        links_html += f"""
        <a href="{bib_entry['source']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">Source</a>
        """
    if bib_entry['supplemental'] != "":
        links_html += f"""
        <a href="{bib_entry['supplemental']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">Supplemental</a>
        """
    if bib_entry['video'] != "":
        links_html += f"""
        <a href="{bib_entry['video']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">Video</a>
        """

    if bib_entry['press'] != "":
        links_html += f"""
        <a href="{bib_entry['press']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">Press Release</a>
        """
        
    if bib_entry['bibtex'] != "":
        links_html += f"""
        <a href="{bib_entry['bibtex']}" class="btn btn-sm z-depth-0" role="button" target="_blank" style="font-size:12px;">BibTex</a>
        """
    
    return links_html

In [287]:
def create_lineitems():
    
    with open(bib_file) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)

    with open('Pub_Links.txt') as links_file:
        links_database = bibtexparser.load(links_file)
    
    pubs = ''
    pres = ''
        
    for i in range(len(bib_database.entries)):

        author_list = []
        for author in bib_database.entries[i]['author'].split(' and '):
            names = list(reversed(author.split(', ')))
            final = " ".join(names).replace("{","").replace("}","").replace(",","").replace("~"," ").replace('\\', '').replace('|', '')
            final = final.replace("'e", "é").replace("'a", "á").replace('"U', 'Ü')
            author_list.append(final)

        if len(author_list) > 5:
            if any('Carreira' in s for s in author_list[0:5]):
                bib_database.entries[i]['author'] = ', '.join(author_list[0:5]) + ' et al.'
            elif any('Carreira' in s for s in author_list[6:]):
                idx = [idx for idx, s in enumerate(author_list) if 'Carreira' in s][0]
                bib_database.entries[i]['author'] = ', '.join(author_list[0:3]) + ' et al., including ' + author_list[idx]
        else:
            bib_database.entries[i]['author'] = ', '.join(author_list)
        
        bib_database.entries[i]['title'] = bib_database.entries[i]['title'].replace("{","").replace("}","")

        for j in range(len(links_database.entries)):
            if links_database.entries[j]['ID'] == bib_database.entries[i]['ID']:
                links_html = create_buttons(links_database.entries[j])

        if bib_database.entries[i]['ENTRYTYPE'] == 'article':

            if bib_database.entries[i]['journal'] == '\\apj':
                bib_database.entries[i]['journal'] = 'The Astrophysical Journal'

            pubs += rf"""
<li>
<div class="pub-row">

  <div class="col-sm-3 abbr" style="position: relative;padding-right: 15px;padding-left: 15px;">
    <img src="assets/img/{bib_database.entries[i]['ID']}.png" class="teaser img-fluid z-depth-1">
  </div>

  <div class="col-sm-9" style="position: relative;padding-right: 15px;padding-left: 20px;">
    <div class="title">{bib_database.entries[i]['title']}</div>
    <div class="author">{bib_database.entries[i]['author']}</div>
    <div class="periodical"><i>{bib_database.entries[i]['journal']}, Volume {bib_database.entries[i]['volume']}, {bib_database.entries[i]['month']} {bib_database.entries[i]['year']}.</i></div>
    <div class="links">
        {links_html}
    </div>
  </div>
</div>
</li>
            """

        elif bib_database.entries[i]['ENTRYTYPE'] == 'inproceedings':
            pres += f"""
<li>
<div class="pub-row">

  <div class="col-sm-3 abbr" style="position: relative;padding-right: 15px;padding-left: 15px;">
    <img src="assets/img/{bib_database.entries[i]['ID']}.png" class="teaser img-fluid z-depth-1">
  </div>

  <div class="col-sm-9" style="position: relative;padding-right: 15px;padding-left: 20px;">
    <div class="title">{bib_database.entries[i]['title']}</div>
    <div class="author">{bib_database.entries[i]['author']}</div>
    <div class="periodical"><i>{bib_database.entries[i]['booktitle']}, {bib_database.entries[i]['month']} {bib_database.entries[i]['year']}.</i></div>
    <div class="links">
        {links_html}
    </div>
  </div>
</div>
</li>
            """
            
        
    return pubs, pres


In [289]:
def create_publications_html(pubs_lineitems):
    
    top = f"""
<h2 id="publications" style="margin: 2px 0px -15px;">Publications</h2>

<div class="publications">
<ol class="bibliography">
    """
    
    bottom = f"""
<br>

</ol>
</div>
    """
    
    final = top + pubs_lineitems + bottom
    
    with open('../_includes/publications.html', 'w') as f:
        f.write(final)

In [291]:
def create_presentations_html(pres_lineitems):
    
    top = f"""
<h2 id="publications" style="margin: 2px 0px -15px;">Presentations</h2>

<div class="publications">
<ol class="bibliography">
    """
    
    bottom = f"""
<br>

</ol>
</div>
    """
    
    final = top + pres_lineitems + bottom
    
    with open('../_includes/presentations.html', 'w') as f:
        f.write(final)

In [293]:
pubs_lineitems, pres_lineitems = create_lineitems()

create_publications_html(pubs_lineitems)
create_presentations_html(pres_lineitems)